In [1]:
#Load packages
from netCDF4 import Dataset
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
import netCDF4 as nc4

sys.path.append("/home2/datahome/tpicard/python/Python_Modules_p3_pyticles/")

from Modules import *
from Modules_gula import *
import torch
from torch.utils.data import DataLoader, Dataset


In [2]:
# Read the data for a given period

def read_data():
    datetime=10*101
    nc = nc4.Dataset('/home2/datawork/tpicard/Pyticles/CNN_DATA/trainning_database_{0:06}.nc'.format(datetime),'r')
    inputs = np.nan_to_num(np.asfortranarray(nc.variables['inputs']))
    inputs= np.transpose(inputs, axes=[0,3,1,2])
    outputs = np.nan_to_num(np.asfortranarray(nc.variables['outputs'])*100) #Pour avoir une somme = 1
    outputs= np.transpose(outputs, axes=[0,3,1,2])
    nc.close()
    return(inputs,outputs)

# Return a set of trainning and test data

def train_test_data():
    inputs,outputs = read_data()
    return(inputs[:90,:,:,:],outputs[:90,:,:,:],inputs[90:,:,:,:],outputs[90:,:,:,:]) # Take only the first 90 samples


In [116]:
# Check the shape of the data
np.isnan(train_images)

array([[[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, Fa

In [3]:
class Pdf_Image_DataSet(Dataset):
    def __init__(self,images, pdf, transform=None):
        
        self.pdf = pdf
        self.images = images
        self.transform = transform

    def __len__(self):
        return self.pdf.shape[0]

    def __getitem__(self, idx):
        # select coordinates
        pdf_sample = self.pdf[idx,:,:,:]
        image_sample = self.images[idx,:,:,:]
        
        if self.transform:
            pdf_sample = self.transform(pdf_sample)
            image_sample = self.transform(image_sample)
            
        return image_sample, pdf_sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        return torch.FloatTensor(sample)

In [14]:
# hyperparameters
batch_size = 10

(train_images,train_pdf,test_images,test_pdf) = train_test_data()

## reduce size dataset
train_set = Pdf_Image_DataSet(train_images,train_pdf,transform= ToTensor())
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=False)

test_set = Pdf_Image_DataSet(test_images,test_pdf,transform= ToTensor())
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=False)

In [5]:
# Check the shape of inputs and outputs

for X, y in test_loader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([2, 52, 400, 400])
Shape of y:  torch.Size([2, 1, 80, 80]) torch.float32


In [6]:
from torch import nn
from torch import optim
import progressbar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
# Achitecture of the model 

class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(    
            nn.Conv2d(52, 26, kernel_size = 1, padding = 0),
            nn.ReLU(True),

            nn.AvgPool2d(5),

            nn.Conv2d(26, 1, kernel_size = 1, padding = 0),
            
            nn.Flatten(start_dim=1, end_dim=- 1), # image 80x80 to a vector
            nn.Softmax(dim=1), 
         )
    def forward(self, z):    
        out = self.cnn(z)
        print(out.shape)
        out = out.view(out.shape[0],80,80)
        return out

In [56]:
def weights_init(m):
    """
    This function initializes the model weights randomly from a 
    Normal distribution. This follows the specification from the DCGAN paper.
    https://arxiv.org/pdf/1511.06434.pdf
    Source: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    #elif classname.find('AvgPool2d') != -1:
    #    nn.init.normal_(m.weight.data, 1.0, 0.02)
    #    nn.init.constant_(m.bias.data, 0)

In [57]:
# create network
model = CNN().to(device)

# weight initialization
model = model.apply(weights_init)

# define loss and optimizers
loss_fn = nn.MSELoss()
lr = 0.0002
optimizer = optim.Adam(model.parameters(),lr= lr, betas=(0.5, 0.999))

In [58]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        
        # TRAIN MODEL
        model.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch%10 == 0:
        print('\nEpoch [{}/{}] -----------------------------------------------------------------------------'
            .format(epoch+1, nb_epoch))

    if batch % 100 == 0:
        loss, current = loss.item(), batch * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
    return(pred,loss)

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            #print(loss_fn(pred, y))
            test_loss += loss_fn(pred, y).item()
    
    test_loss /= num_batches
    print(f"Test Error: Avg loss: {test_loss:>8f} \n") 

In [59]:
# Train the model
nb_epoch = 2
epoch = 0
#loss_train_list=[]
#loss_test_list=[]

for i in progressbar.progressbar(range(nb_epoch)):
    epoch +=1
    (pred,loss) = train_loop(train_loader, model, loss_fn, optimizer)
    print(loss)
    print(pred)
    test_loop(test_loader, model, loss_fn)   


N/A% (0 of 2) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
tensor(5.2691e-06, grad_fn=<MseLossBackward0>)
tensor([[[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         ...,
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001]],

        [[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         ...,
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
 

 50% (1 of 2) |#############             | Elapsed Time: 0:00:21 ETA:   0:00:21

torch.Size([8, 6400])
Test Error: Avg loss: 0.000006 

torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
torch.Size([10, 6400])
tensor(5.7379e-06, grad_fn=<MseLossBackward0>)
tensor([[[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         ...,
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001]],

        [[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
         ...,
         [0.0002, 0.0002, 0.0002,  ..., 0.0001, 0.0001, 0.0001],
         [0

100% (2 of 2) |##########################| Elapsed Time: 0:00:42 Time:  0:00:42


torch.Size([8, 6400])
Test Error: Avg loss: 0.000006 



In [61]:
pred.shape

torch.Size([10, 80, 80])

In [64]:
torch.sum(pred[0,:,:])

tensor(1.0000, grad_fn=<SumBackward0>)